# Assignment 1: NYC Taxi Data

In [1]:
from pyspark.sql import SparkSession

In [2]:
# Create a local spark session
spark = SparkSession.builder \
        .appName('nyc-taxi-sql') \
        .getOrCreate()

In [3]:
# Read parquet file
df = spark.read.load("./output")

In [5]:
df.createOrReplaceTempView("nyc_taxi_data_2017_18")

## Business Questions

### Q1.a. For each year and month: What was the total number of trips?

In [6]:
spark.sql("""
SELECT year
,month
,COUNT(*) AS number_of_trips
FROM nyc_taxi_data_2017_18
GROUP BY year
,month
ORDER BY year, month
""").show(24)

+----+-----+---------------+
|year|month|number_of_trips|
+----+-----+---------------+
|2017|    1|        1064589|
|2017|    2|        1017474|
|2017|    3|        1152610|
|2017|    4|        1076626|
|2017|    5|        1054177|
|2017|    6|         971566|
|2017|    7|         910004|
|2017|    8|         862947|
|2017|    9|         878115|
|2017|   10|         921352|
|2017|   11|         869909|
|2017|   12|         901582|
|2018|    1|         789266|
|2018|    2|         766246|
|2018|    3|         832798|
|2018|    4|         796114|
|2018|    5|         793394|
|2018|    6|         735642|
|2018|    7|         681211|
|2018|    8|         663117|
|2018|    9|         663566|
|2018|   10|         707061|
|2018|   11|         653351|
|2018|   12|         681425|
+----+-----+---------------+



### Q1.b. For each year and month: Which weekday had the most trips?

In [13]:
spark.sql("""
SELECT year
    ,month
    ,pickup_weekday
    ,total_trips
FROM (SELECT year
        ,month
        ,DATE_FORMAT(pickup_datetime, "EEEE") AS pickup_weekday
        ,COUNT(*) AS total_trips
        ,ROW_NUMBER() OVER (PARTITION BY year,month ORDER BY COUNT(*) DESC) AS row_num
    FROM nyc_taxi_data_2017_18
    GROUP BY year
        ,month
        ,pickup_weekday
    )
WHERE row_num = 1
""").show()

+----+-----+--------------+-----------+
|year|month|pickup_weekday|total_trips|
+----+-----+--------------+-----------+
|2017|    3|        Friday|     218310|
|2017|    8|      Thursday|     137980|
|2017|   10|        Sunday|     149288|
|2018|   10|     Wednesday|     114659|
|2018|    1|     Wednesday|     130280|
|2018|    3|      Saturday|     156004|
|2018|    8|        Friday|     118794|
|2017|    7|      Saturday|     176039|
|2018|    5|      Thursday|     132998|
|2017|   12|        Friday|     164254|
|2017|    9|      Saturday|     173461|
|2017|    4|      Saturday|     218381|
|2018|    9|      Saturday|     121766|
|2018|    7|       Tuesday|     107744|
|2018|   12|      Saturday|     118918|
|2018|   11|        Friday|     116664|
|2018|    6|      Saturday|     133693|
|2017|    2|      Saturday|     183968|
|2017|   11|      Thursday|     144139|
|2017|    5|      Saturday|     171600|
+----+-----+--------------+-----------+
only showing top 20 rows



### Q1.c. For each year and month: What was the average number of passengers?

### Q1.d. For each year and month: What was the average amount paid per trip (total_amount)?

### Q1.d. For each year and month: What was the average amount paid per passenger (total_amount)?

### Q2.a. For each taxi colour (yellow and green): What was the average, median, minimum and maximum trip duration in seconds?

### Q2.b. For each taxi colour (yellow and green): What was the average, median, minimum and maximum trip distance in km?

### Q2.c. For each taxi colour (yellow and green): What was the average, median, minimum and maximum speed in km per hour?

### Q2.d. For each taxi colour (yellow and green): What was the percentage of trips where the driver received tips?

### Q3. For trips where the driver received tips, What was the percentage where the driver received tips of at least $10.

### Q4.a. For each duration bin calculate: Average speed (km per hour)
Bins are Under 5 Mins, From 5 mins to 10 mins, From 10 mins to 20 mins, From 20 mins to 30 mins, At least 30 mins:

### Q4.b. For each duration bin calculate: Average distance per dollar (km per $)
Bins are Under 5 Mins, From 5 mins to 10 mins, From 10 mins to 20 mins, From 20 mins to 30 mins, At least 30 mins.

Assuming US Dollars

### Q5. Which duration bin will you advise a taxi driver to target to maximise his income?